# 1.1 言語モデルとは

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-small")
tokenizer = AutoTokenizer.from_pretrained("cyberagent/open-calm-small")

In [2]:
input = tokenizer("東京は日本の", return_tensors="pt")

In [3]:
for i in input["input_ids"][0]:
    print(i, tokenizer.decode(i))

tensor(1136) 東京
tensor(33558) は日本の


In [4]:
tokens = model.generate(**input, max_new_tokens=1, do_sample=False)
tokens

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


tensor([[ 1136, 33558,  6338]])

In [5]:
print("length: {}".format(len(tokens[0])))
for i in tokens[0]:
    print(i, tokenizer.decode(i))

length: 3
tensor(1136) 東京
tensor(33558) は日本の
tensor(6338) 首都


In [6]:
out = model.generate(
    **input,
    max_new_tokens=1,
    return_dict_in_generate=True,
    output_scores=True,
)
out.keys()
out["scores"][0].shape

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


torch.Size([1, 52096])

In [7]:
top5 = torch.topk(out["scores"][0][0], 5)
top5

torch.return_types.topk(
values=tensor([18.5860, 16.7091, 16.6892, 16.5491, 16.4221]),
indices=tensor([6338, 4303,  530, 2004,  308]))

In [8]:
for i in range(5):
    print(i+1, tokenizer.decode(top5.indices[i].item()), top5.values[i].item())

1 首都 18.585966110229492
2 未来 16.709089279174805
3 最 16.689170837402344
4 文化 16.549089431762695
5 「 16.422061920166016


# 1.2 言語モデルとチャットボット

In [9]:
input = tokenizer("日本の首都はどこですか?", return_tensors="pt")
tokens = model.generate(**input, max_new_tokens=50, do_sample=False)
tokens.shape

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


torch.Size([1, 55])

In [10]:
print(tokens[0])
tokenizer.decode(tokens[0], skip_special_tokens=True)

tensor([ 3296,  6338, 20935,  3726,    32,   186,   308,  1136,   309,   495,
         2200,   258,   245,  3960,   308,  1136,   309,  4387,   247,  2811,
          245,  1136,   495,  2200,   258,   245,  1136,   495,  2200,   258,
          245,  1136,   495,  2200,   258,   245,  1136,   495,  2200,   258,
          245,  1136,   495,  2200,   258,   245,  1136,   495,  2200,   258,
          245,  1136,   495,  2200,   258])


'日本の首都はどこですか?\n「東京」という都市が、なぜ「東京」なのか。それは、東京という都市が、東京という都市が、東京という都市が、東京という都市が、東京という都市が、東京という都市が、東京という都市が'

In [11]:
input = tokenizer("今日は天気がよいですね\n"+"そうですね\n"+"どこかへいきましょうか。", return_tensors="pt")
tokens = model.generate(**input, max_new_tokens=20, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [12]:
tokenizer.decode(tokens[0], skip_special_tokens=True)

'今日は天気がよいですね\nそうですね\nどこかへいきましょうか。\nさて、\n今日は、\n「  いい天気 」\nです。\nいい天気ですね'

# 1.3 日本語特化のLLM

## Llama2 by Meta
https://llama.meta.com/

## OpenCALM by CyberAgent
https://huggingface.co/cyberagent

## japanese-large-lm by LINE
https://huggingface.co/line-corporation/japanese-large-lm-3.6b

## rinna GPT by rinna
https://huggingface.co/rinna/japanese-gpt-neox-3.6b


# 1.4 LLMの利用

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_id = "cyberagent/open-calm-small"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
out = generator("東京は日本の", max_new_tokens=50, do_sample=False)
print(out[0])

{'generated_text': '東京は日本の首都である。\n\n歴史\n古代には、古代のローマ・カトリック教会は、ローマ・カトリック教会のローマ教会と、ローマ・カトリック教会のローマ教会と、ローマ・カトリック教会のローマ教会と、ローマ・カトリック教会のローマ教会と、ローマ'}


In [14]:
input = tokenizer("東京は日本の", return_tensors="pt")
tokens = model.generate(**input, max_new_tokens=50)
output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)

東京は日本の首都である。

歴史
古代には、古代のローマ・カトリック教会は、ローマ・カトリック教会のローマ教会と、ローマ・カトリック教会のローマ教会と、ローマ・カトリック教会のローマ教会と、ローマ・カトリック教会のローマ教会と、ローマ


In [15]:
print(tokenizer.encode("。"))

[247]


In [16]:
input = tokenizer("東京は日本の", return_tensors="pt")
tokens = model.generate(**input, max_new_tokens=30,
                        eos_token_id=tokenizer.encode("。"),
                        pad_token_id=tokenizer.pad_token_id,
                        do_sample=True,
                        num_return_sequences=5)
for i in range(5):
    output = tokenizer.decode(tokens[i], skip_special_tokens=True)
    print(output)

東京は日本の首都であり、東京は日本の首都です。
東京は日本のトップクラスの大学です。
東京は日本の「夢」です。
東京は日本の「食」にかかわるすべての業界を応援していきます。
東京は日本の首都である。
